Processing Database

In [2]:
import src.notion_API as notion
import src.pocketbase_API as pocket
from typing import  Callable

In [3]:
def get_all(getFunction: Callable[[dict, bool], any], params: dict):

    all_params = {'perPage': 500}
    all_params.update(params)
    first = getFunction(all_params, full=True).json()
    items = first['items']
    for page in range(1,first['totalPages']):
        all_params.update({'page': (page+1)})
        output = getFunction(all_params, full=True).json()
        items = items + output['items']

    return items

In [14]:
def update_action_counts():

    # Run Times
    #   3m 25.9s (first run, no improvements)
    #   3m 13.5s (second rum, no improvements)
    #   3m 3.9s (first run, added #2)
    #   2m 59.9s (second run ,added #2)
    #   3m 28.9s (first run, added #1,#2)

    # Lets run some performance tests

    # Do we need to
    #   1. not use a for-loop -> map?(No), list comprehension(slower)
    #   2. use Python's filter function to get counts?
    def count_iterable(i):
        return sum(1 for e in i)
    #   3. Batch the updates
    #   4. check when the last update was (ID of what day) and update counts on days greater than last update
    
    # Get all actions (This will need to be changed as run time is 3m 25.9s)
    actions = get_all(pocket.get_actions,{})
    # Get all days
    days = get_all(pocket.get_days,{})

    # [pocket.patch_action(id=action['id'],json={"total_count": count_iterable(filter(lambda y: y['action'] == action['id'], days))}) for action in actions]

    # Loop through all actions and filter days by their action IDs. (Filter in Python?)
    for action in actions:
        id = action['id']
        name = action['action']
        # action_count = len(get_all(pocket.get_days,{'filter': f'action="{id}"'}))
        action_count = count_iterable(filter(lambda x: x['action'] == id, days))
        pocket.patch_action(id=id,json={"total_count": action_count})


In [15]:
update_action_counts()

In [12]:
old_text = "’"
old_actions = pocket.get_actions(params={'filter': f'action~"{old_text}"'},full=False)
new_text = "'"
new_actions = pocket.get_actions({'filter': f'action~"{new_text}"'},full=False)

for key in list(old_actions.keys()):
    old_id = old_actions[key]
    new_id = new_actions[key.replace(old_text, new_text)]
    days = pocket.get_days(params={'filter': f'action="{old_id}"'}, full=True).json()['items']
    for day in days:
        pocket.patch_day(id=day['id'],json={'action': new_id})

In [20]:
level_one = pocket.get_levels('level_one',params={},full=False)

In [28]:
breakdown = {}
for cat in list(level_one.keys()):
    breakdown[cat] = sum(map(lambda x: x['total_count'], get_all(pocket.get_actions,params={'filter': f'level_one.classification="{cat}"'})))

In [23]:
pocket.get_days(params={'filter': 'level_one="y17f3rgi6ttkw49"'}).json()

{'code': 400, 'message': 'Invalid filter parameters.', 'data': {}}

In [29]:
breakdown

{'Sleep': 5119, 'Work': 5894, 'Life': 3903, 'Food': 770, 'Unknown': 1782}